This tutorial can be found and ran in the GITHUB libray corrAI: https://github.com/BuildingEnergySimulationTools/corrai

---
# Characterisation method tutorial

The aim of this tutorial is to provide tools for identification and optimisation of parameters of a model, using **CorrAI** modules. It contains three parts:

1. **Model definition**: short definition of a mathematical model of an opaque wall, using a resistance/capacity approach.

2. **Measurement import**: Loading of measurements performed on a test benchto serve as reference data for comparison with the model.

3. **Model identification**: Identificaiton of the parameters that need to be adjusted to improve the model's accuracy. This involves fitting the model to the data and refining its parameters. A sensitivity analysis is performed in tutorial "Sensitivity analysis of an opaque wall model" to rank the parameters by order of influence on the model error.
---

# 1. Model definition

Measurement of temperature evolution in layers of an opaque wall installed on a real-scale test bench were performed. The aim is to identify the thermal conductivity of the wall, based on a simplified model.

For more detailed on the model description and measurement, see description in tutorial "Sensitivity analysis of an opaque wall model".

<div style="text-align: center;">
  <img src="images/RC_model.png"  style="height:400px;"> <br>
  <em>Figure : RC model</em>
</div>


As a reminder, it should follow the `Model` interface define in corrai, which guarantees that it can be used in calibration, sensitivity analysis, and optimization workflows.

A valid simulator must implement the following concepts:

- **`simulate(property_dict, simulation_options, simulation_kwargs)`**
  Main entry point. Runs the simulation and returns results as a `pandas.DataFrame` indexed by time.

- **`property_dict`**
  Dictionary of model parameters `{property_name: value}`.
  These overwrite default values to perform calibration, parameter sweeps, or optimization.
Here, the wall is described by default physical properties, which can be overridden at runtime :

In [ ]:
import numpy as np
import pandas as pd
from corrai.base.model import Model

class OpaqueWallSimple(Model):
    def simulate(
        self,
        property_dict: dict,
        simulation_options: dict,
        simulation_kwargs: dict | None = None,
    ) -> pd.DataFrame:

        default_parameters = {
            "R_ext": 0.005,
            "R_int": 0.01,
            "R_concrete": 0.10,
            "R_ins": 0.32,
            "C_concrete": 2.95e6,
            "C_ins": 3.64e4,
            "alpha": 0.5,
            "S_wall": 7,
            "epsilon": 0.4,
            "fview": 0.5,
        }
        parameters = {**default_parameters, **property_dict}

        R_ext       = parameters["R_ext"]
        R_int       = parameters["R_int"]
        R_concrete  = parameters["R_concrete"]
        R_ins       = parameters["R_ins"]
        C_concrete  = parameters["C_concrete"]
        C_ins       = parameters["C_ins"]
        alpha       = parameters["alpha"]
        S_wall      = parameters["S_wall"]
        epsilon     = parameters["epsilon"]
        fview       = parameters["fview"]

        sigma = 5.67e-8  # W/m^2/K^4

        df = simulation_options["dataframe"]
        time  = df["time_sec"].values
        T_ext = df["T_ext"].values
        T_int = df["T_int"].values
        Q_rad = df["Pyr"].values

        startTime = simulation_options.get("startTime", time[0])
        stopTime  = simulation_options.get("stopTime",  time[-1])

        mask  = (time >= startTime) & (time <= stopTime)
        time  = time[mask]
        T_ext = T_ext[mask]
        T_int = T_int[mask]
        Q_rad = Q_rad[mask]

        # init
        T_se        = np.zeros(len(time))
        T_concrete  = np.zeros(len(time))
        T_ins       = np.zeros(len(time))
        T_interface = np.zeros(len(time))
        T_si        = np.zeros(len(time))
        T_sky       = np.zeros(len(time))

        T_se[0]        = T_ext[0]
        T_concrete[0]  = 299 - 273.15
        T_ins[0]       = T_int[0]
        T_interface[0] = (T_ins[0] + T_concrete[0]) / 2
        T_si[0]        = T_int[0]
        T_sky[0]       = T_int[0]

        for t in range(1, len(time)):
            dt = time[t] - time[t - 1]

            T_sky[t] = 0.0552 * (T_ext[t] ** 1.5)

            Q_rad_sky = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_sky[t] ** 4) * S_wall
            Q_rad_amb = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_ext[t - 1] ** 4) * S_wall
            Q_rad_dir = Q_rad[t - 1] * alpha * S_wall

            T_se[t] = (
                T_ext[t - 1] / R_ext
                + T_ins[t - 1] / (R_ins / 2)
                + Q_rad_dir - Q_rad_sky - Q_rad_amb
            ) / (1 / R_ext + 1 / (R_ins / 2))

            T_interface[t] = (
                T_ins[t - 1] / (R_ins / 2) + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / (R_concrete / 2) + 1 / (R_ins / 2))

            T_si[t] = (
                T_int[t - 1] / R_int + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / R_int + 1 / (R_concrete / 2))

            T_ins[t] = T_ins[t - 1] + dt / C_ins * (
                (T_se[t] - T_ins[t - 1]) / (R_ins / 2)
                + (T_interface[t] - T_ins[t - 1]) / (R_ins / 2)
            )

            T_concrete[t] = T_concrete[t - 1] + dt / C_concrete * (
                (T_interface[t] - T_concrete[t - 1]) / (R_concrete / 2)
                + (T_si[t] - T_concrete[t - 1]) / (R_concrete / 2)
            )

        # output
        df_out = pd.DataFrame(
            {
                "T_concrete":  T_concrete,
                "T_interface": T_interface,
                "T_ins":       T_ins,
            },
            index=df.index[mask],
        )
        self.simulation_options = simulation_options
        return df_out

# 2. Measurement import
Let's now load  the reference cell measurement data on python that will be used as boundary conditions. Note that although the data loaded here should have be cleaned beforhand, it is always important to check it before using it for analyses.

## Loading the dataframe

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/tuto_data_SA.csv",
    index_col=0,
    sep=";",
    decimal=".",
    parse_dates=True
)

In [ ]:
reference_df.head()

In [ ]:
reference_df.loc[:,"T_ins"] = reference_df['T_ins_2'] # middle of insulation temperature
reference_df.loc[:,"T_int"] =reference_df[['Tint_1', 'Tint_2']].mean(axis=1) # indoor temperature
reference_df.loc[:,"T_interface"] =  reference_df[['T_interface_1', 'T_interface_2']].mean(axis=1) # interface temperature, between insulation and concrete panels

## Reference simulation

Let's run our first simulation with default parameters, on a period with both sunny and cloudy days.
To create a "time_sec" column used by the model, we can use the following function:

In [ ]:
import datetime as dt

def datetime_to_seconds(index_datetime):
    time_start = dt.datetime(index_datetime[0].year, 1, 1, tzinfo=dt.timezone.utc)
    new_index = index_datetime.to_frame().diff().squeeze()
    new_index.iloc[0] = dt.timedelta(
        seconds=index_datetime[0].timestamp() - time_start.timestamp()
    )
    sec_dt = [elmt.total_seconds() for elmt in new_index]
    return list(pd.Series(sec_dt).cumsum())

In [ ]:
feat_train = reference_df.loc["2024-09-04 00:00":"2024-09-07 00:00"]
feat_train["time_sec"] = datetime_to_seconds(feat_train.index)

sim_opt ={
    "dataframe":feat_train,
    "startTime":feat_train["time_sec"].iloc[0],
    "endTime": feat_train["time_sec"].iloc[-1],
}

In [ ]:
model = OpaqueWallSimple()
init_res = model.simulate(
    property_dict ={},
    simulation_options=sim_opt
)

Let's compare results of simulation to measurement:

In [ ]:
init_res_renamed = init_res.copy()
init_res_renamed.index = init_res_renamed.index.tz_localize(None)
init_res_renamed = init_res_renamed.rename(columns={
    "T_concrete": "T_concrete_PYTHON",
    "T_interface": "T_interface_PYTHON",
    "T_ins": "T_insulation_PYTHON",
})
measure_comp = pd.concat([
    feat_train[["T_interface", "T_ins"]],
    init_res_renamed[["T_interface_PYTHON", "T_insulation_PYTHON" ]]], axis = 1)

color_map = {
    "T_interface": "darkblue",
    "T_interface_PYTHON": "blue",
    "T_ins": "darkgreen",
    "T_insulation_PYTHON": "green"
}

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Interface layer", "Insulation layer"))

color_map = {
    "T_interface": "darkblue",
    "T_interface_PYTHON": "blue",
    "T_ins": "darkgreen",
    "T_insulation_PYTHON": "green"
}
for col in ["T_interface", "T_interface_PYTHON"]:
    fig.add_trace(
        go.Scatter(x=measure_comp.index, y=measure_comp[col], mode="lines",
                   name=col, line=dict(color=color_map[col])),
        row=1, col=1
    )
for col in ["T_ins", "T_insulation_PYTHON"]:
    fig.add_trace(
        go.Scatter(x=measure_comp.index, y=measure_comp[col], mode="lines",
                   name=col, line=dict(color=color_map[col])),
        row=1, col=2
    )
fig.update_layout(
    title="Comparaison Python vs Measurement",
    width=1000,
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5)
)

fig.show()

# 3. Identification
Now, we proceed to finding optimal values for these parameters by minimizing the coefficient of variation of root mean square error (cv_rmse) between one or several measured nodes, and one or several relevant outputs of our model.

**Step-by-Step Process**

1. **Define the model and parameters**: by defining `OpaqueWallSimple` (this we already did) and specifying the parameters to be identify. Each parameter includes a name, an interval of possible values, a type, and an initial value.

2. **Instantiate an objective function**: We create an instance of the `ObjectiveFunction` class, providing the model, simulation options, list of parameters, and indicators. The `scipy_obj_function` method of `ObjectiveFunction` will be used as the objective function for optimization. This method calculates for instance the cv_rmse of measured vs simulated temperatures, for the given parameter values.

3. **Perform optimization**:  We use minimization functions (for instance `minimize_scalar`  from `scipy.optimize`) to minimize the objective function.


## Definition of an objective function
In parameter optimization, we aim to adjust certain model parameters to minimize the difference between simulated and observed data. The objective function is a scalar function that quantifies this difference. In this case, the CV_RMSE (Coefficient of Variation of Root Mean Square Error) is used as a measure of how well the model output matches the reference measurements.

**How the ObjectiveFunction Class Works** : The ObjectiveFunction class simplifies the process of optimizing model parameters by providing a structured way to:

- Run simulations: For a given set of parameters, the model is simulated over the input data.
- Calculate error metrics (e.g., CV_RMSE): The model output is compared to the reference measurements, and a scalar error metric is calculated (such as the CV_RMSE).
- Optimize: The class can then be used with optimization algorithms (like scipy.optimize or pymoo) to adjust the model parameters in order to minimize the error metric.


To do so, the  `ObjectiveFunction` class is designed to facilitate the optimization of model parameters using `scipy.optimize` or `pymoo` optimization methods by encapsulating the logic for simulation and indicator calculation. The `ObjectiveFunction` class takes a model, simulation options, a list of parameters to be calibrated, and a list of indicators as input. It provides methods to calculate the objective function, which can be used by optimization routines to find the optimal parameters.

### Attributes

- **model**: The model to be calibrated.
- **simulation_options**: A dictionary containing simulation options, including input data.
- **param_list**: A list of dictionaries specifying the parameters to be calibrated.
- **indicators_config**: A dictionary where the keys are the names of the indicators corresponding to the model outputs, and the values are either:
    - An aggregation function to compute the indicator (e.g., np.mean, np.sum).
    - A tuple (function, reference_data) if a comparison with reference data is required (e.g., sklearn.metrics.mean_squared_error, corrai.metrics.mae).
- **scipy_obj_indicator**: The name of the indicator used as the objective function for optimization with scipy.optimize. By default, it is the first key in indicators_config.

## Definition of parameters
Each decision variable of the optimization problem must be described using a `Parameter` object.
A parameter specifies:
* `name` — The variable name (string, must be unique within the problem).
* `ptype` — Variable type, one of:
    * `"Real" `— Continuous real number
    * `"Integer"` — Discrete integer
    * `"Binary"` — Boolean, domain {False, True} (set automatically if no domain is given)
    *` "Choice"` — Categorical variable with a fixed set of discrete options
* `Domain definition` — Choose exactly one of:
    * ` interval=(lo, hi) `— Lower and upper bounds (for "Real" and "Integer", optional for "Binary" if you want (0,1))
    * ` values=(v1, v2, …)` — Explicit list/tuple of allowed values (for "Choice", and optionally for "Integer", "Real", or "Binary")
*` Optional fields`:
    * `init_value` — Initial value (or tuple/list of initial values for batch runs); must be within the defined domain
    * `relabs` — `"Absolute"` or `"Relative"`
    * `model_property` — String or tuple specifying the corresponding property in the simulation/model
    * `min_max_interval` — Optional extra bounds for use in analysis/validation

## Application: One-dimensional optimization problem

First, we can try scalar functions optimization from scipy (different methods: brent, boulded, golden ... see documentation on scipy website).
For Scypi, each objective function is minimized for optimization:
- Here we chose as indicators the temperature calculated and measured within the wall insulation. Note this could be another node (a heat flux densitiy, another temperature node).
- The identified parameter here is  $alpha$.

Let's define the parameter as Real, between 0 and 1, with a default value of 0.5. By default, it will be considered as an absolute value.

In [ ]:
from corrai.base.parameter import Parameter

calibration_params = Parameter(
    name='alpha',
    interval=(0,1),
    ptype="Real",
    model_property='alpha'
)

We can now instanciate an objective function, using `ObjectiveFunction`.

In [ ]:
from corrai.optimize import ObjectiveFunction
from corrai.base.metrics import cv_rmse

obj_func = ObjectiveFunction(
    model=OpaqueWallSimple(),
    simulation_options=sim_opt,
    parameters=[calibration_params],
    indicators_config={"T_ins": (cv_rmse, feat_train["T_ins"])},
)

Let's see how ObjectiveFunction works by calculating the objective function value for a given value of alpha:

In [ ]:
obj_func.function(
    parameter_value_pairs = [(calibration_params, 0.5)]  # Example value for alpha
)

The `minimize_scalar` function in `scipy.optimize` is used for scalar function minimization, specifically for one-dimensional optimization problems. This function finds the minimum value of a scalar function over a specified interval. The main methods are `Brent`, `bounded`, and `golden`.

The function returns an optimization result object that contains information about the optimization process and the final solution.

- **Brent** :The Brent method uses Brent’s algorithm, which combines a parabolic interpolation with the golden section search. This method does not require the interval bounds.
- **Golden** : Employs the golden section search method, which reduces the interval of uncertainty using the golden ratio. Simple and reliable for unimodal functions, but may be slower than Brent's method.
- **Bounded** : Restricts the search to the specified bounds using a combination of golden section search and parabolic interpolation.
Advantages: Ensures that the solution remains within the given bounds, making it ideal for constrained problems.

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

result = minimize_scalar(
    obj_func.scipy_obj_function, 
    bounds=obj_func.bounds[0],
    method="Bounded"
)

result

A solution is found with a value of 0.12 for alpha and 4.36 for the CV_RMSE. Let's check if the parameter value is close to the boundaries.

In [ ]:
obj_func.bounds

It is indeed not far from 0.1 but not a the limit. Let's now run the simulation using this parameter value and compare with the initial simulation.

In [ ]:
parameter_names = ["alpha"]
parameter_dict1 = {param_name: result.x for i, param_name in enumerate(parameter_names)}

result_optim = model.simulate(
    property_dict=parameter_dict1,
    simulation_options=sim_opt
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=feat_train.index,
    y=feat_train["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.add_trace(go.Scatter(
    x=init_res.index,
    y=init_res["T_ins"],
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Initial results"
))

fig.add_trace(go.Scatter(
    x=result_optim.index,
    y=result_optim["T_ins"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="T_insulation - Optimization results"
))


fig.update_layout(
    title='Optimization vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

Results are closer to measurements but still far off.

## Application: multi- objectives and multi parameters optimization

Let's use Pymoo, integrated into the `Problem` class of `CorrAI`  and multi-parameters optimization with multi_objectives.

Note that :
- All **objectives are minimized**.
- All **constraints** must be provided in the form **g(x) ≤ 0** (inequalities).

`Problem` aggregates all evaluator outputs into a single dictionary and then **extracts**:
- **Objectives** in the order of `objective_ids` → matrix **F**
- **Constraints** in the order of `constraint_ids` → matrix **G** (≤ 0)

`Problem` takes as arguments:
- **`parameters`**: as defined earlier from the class `Parameter`
- **`evaluators`**  (or objective functions)
- **`objective_ids`** :   Names (keys) to extract from evaluator outputs to build **F** (in order).
- **`constraint_ids`**  : Optional names to extract for **G** (inequalities ≤ 0). If omitted, `G` is empty.


###  Several parameters, one objective

In [ ]:
# Surface of the tested wall
S_wall =  7

# Thickness of layer
ep_concrete = 0.200 #m
ep_ins = 0.140 #m

# Conductivity of layer
lambda_concrete = 0.13 # (W/mK)
lambda_ins = 0.031 # W/(mK)

# Density of layer
rho_concrete = 2400 # kg/m3
rho_ins = 32.5  # kg/m3"

sc_concrete = 880 # J/kg.K
sc_ins = 1000 # J/kg.K

# solar paremetesr
alpha = 0.2 # absorption coefficient
epsilon = 0.8 # emissivity
fview = 0.5 # view factor of tested wall

Let's define now a list of two parameters, with one as a relative value.
The specified interval defines the relative search space around the initial value.
If no initial value is provided, the model should have a `get_property_value()` method to retrieve a default value.

In [ ]:
calibration_params = [
    Parameter(
        name='R_ext',
        init_value= 0.04/S_wall,
        interval=(0.4, 1.6), # search between 40% and 160% of initial value
        relabs="Relative",
        ptype="Real",
        model_property='R_ext',
    ),
    Parameter(
        name='alpha',
        interval=(0.2, 0.8),
        ptype="Real",
        model_property='alpha',
    ),
]

Here, we can add the interferace temperature (between insulation and concrete panels) as an indicator.

In [ ]:
from corrai.base.metrics import cv_rmse

obj_func = ObjectiveFunction(
    model=OpaqueWallSimple(),
    simulation_options=sim_opt,
    parameters=calibration_params,
    indicators_config={
        "T_ins": (cv_rmse, feat_train["T_ins"]),
        "T_interface": (cv_rmse, feat_train["T_interface"]),
    },
    scipy_obj_indicator="T_ins",
)

Now let's instanciate the problem using the classe `Problem`:

In [ ]:
from corrai.optimize import Problem

problem = Problem(
    parameters=calibration_params,
    evaluators=[obj_func],
    objective_ids=["T_ins", "T_interface"],
)

For a two objective problem, we choose here **NSGA2**, as a well-known multi-objective optimization algorithm based on non-dominated sorting and crowding.
List of algorithms here https://pymoo.org/algorithms/list.html#nb-algorithms-list.

If the verbose=True, some printouts during the algorithm’s execution are provided. This can very from algorithm to algorithm. Here, we execute NSGA2 on a problem where pymoo has no knowledge about the optimum. Each line represents one iteration. The first two columns are the current generation counter and the number of evaluations so far. For constrained problems, the next two columns show the minimum constraint violation (cv (min)) and the average constraint violation (cv (avg)) in the current population. This is followed by the number of non-dominated solutions (n_nds) and two more metrics which represents the movement in the objective space.

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

In [ ]:
algorithm = NSGA2(
    pop_size=50,
    #n_offsprings=10,
    #sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

Here, we will run 15 generation with a population of 50.

In [ ]:
termination = get_termination("n_gen", 15)

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

We can now visualize the objectives functions results using the `Scatter` function from pymoo.

In [ ]:
from pymoo.visualization.scatter import Scatter
Scatter().add(res.F).show()

For a bi-objective problem,and helping us chosing the best set of parameters value, we can use the decomposition method called Augmented Scalarization Function (ASF), a well-known metric in the multi-objective optimization literature.
Let us assume the are equally important by setting the weights to 0.5 and 0.5 and setting these

In [ ]:
from pymoo.decomposition.asf import ASF
F = res.F
approx_ideal = F.min(axis=0)
approx_nadir = F.max(axis=0)
nF = (F - approx_ideal) / (approx_nadir - approx_ideal)

fl = nF.min(axis=0)
fu = nF.max(axis=0)
weights = np.array([0.5, 0.5])
decomp = ASF()

i = decomp.do(nF, 1/weights).argmin()

parameter_names = [param.name for param in calibration_params]
parameter_dict = {param_name: res.X[i][j] for j, param_name in enumerate(parameter_names)}

print(
    "Best regarding ASF: Point \ni = %s\nF = %s" % (i,  F[i]),
    parameter_dict
)


The estimated parameters are consistent with our expectations but at the limits of our bounds. This shows the model is not either adapted to physical behaviour of our system, or boundaries are not large enough. Either way, we can compare the profile of measured indoor temperature with the output that the model predicts given the identified optimal parameters.

Since we used relative parameters, we need to create a list of parameter-value pairs to run the simulation and use the method `simulate_parameter` from Model(ABC), which will convert the given percentage found by the algorithm into an absolute value for R_ext.

In [ ]:
parameter_value_pairs = [
    (param, res.X[i][j])
    for j, param in enumerate(calibration_params)
]

In [ ]:
result_optim = model.simulate_parameter(
    parameter_value_pairs=parameter_value_pairs,
    simulation_options=sim_opt,
)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("T_insulation", "T_interface"))

fig.add_trace(go.Scatter(
    x=init_res.index,
    y=feat_train["T_ins"],
    mode="lines",
    line_color="green",
    name="Measurement"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=init_res.index,
    y=init_res["T_ins"],
    mode="lines",
    line_color="orange",
    name="Initial results"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=feat_train.index,
    y=result_optim["T_ins"],
    mode="lines",
    line_color="brown",
    name="Optimization results"
), row=1, col=1)

# --- Droite : T_interface ---
fig.add_trace(go.Scatter(
    x=init_res.index,
    y=feat_train["T_interface"],
    mode="lines",
    line_color="green",
    name="Measurement"
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=init_res.index,
    y=init_res["T_interface"],
    mode="lines",
    line_color="orange",
    name="Initial results"
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=feat_train.index,
    y=result_optim["T_interface"],
    mode="lines",
    line_color="brown",
    name="Optimization results"
), row=1, col=2)

# Layout
fig.update_layout(
    title="Optimization vs. Measurement",
    xaxis_title="Date",
    yaxis_title="Temperature [°C]",
    showlegend=True
)

fig.show()


#### Validation set
An important step is to check identified parameters on validation set. Let's try on an new period using the last identified values.

In [ ]:
validation_set = reference_df.loc["2024-09-08 00:00":"2024-09-14 00:00"]
validation_set.loc[:,"time_sec"] = datetime_to_seconds(validation_set.index)

validation_set = validation_set.resample('5min').mean()
second_index = datetime_to_seconds(validation_set.index)

new_sim_opt={
    "dataframe":validation_set,
    "startTime": second_index[0],
    "endTime": second_index[-1],  
}

In [ ]:
validation_results = model.simulate_parameter(
    parameter_value_pairs=parameter_value_pairs,
    simulation_options=new_sim_opt,
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=validation_results.index,
    y=validation_results["T_ins"] ,
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Validation result"
))

fig.add_trace(go.Scatter(
    x=validation_results.index,
    y=validation_set["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.update_layout(
    title='Simulation vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

In [ ]:
cv_rmse(
    validation_results["T_ins"],
    validation_set["T_ins"]
)

As we can see, the results are not very good. We could try to identifiy more parameters, or change the model to better represent the physical phenomena.

## 4.3.3 Mixed parameters

In some calibrations, some decision variables are discrete rather than purely continuous—e.g., on/off features (booleans), choices among a few valid modes, or integers for counts. Standard continuous optimizers used earlier assume a smooth search space and therefore won’t handle these variables correctly.

To reflect realistic design decisions and configuration toggles, we now introduce a model with a boolean switch (e.g., enabling/disabling a physical effect) and restrict alpha to a small set of admissible values. This mixed-variable setup better captures practical constraints and uncertainty (e.g., unknown presence of a phenomenon, or controller setpoint options), and requires a mixed-variable optimization strategy.

In [ ]:
import numpy as np
import pandas as pd
from corrai.base.model import Model

class OpaqueWallBool(Model):
    def simulate(
        self,
        property_dict: dict,
        simulation_options: dict,
        simulation_kwargs: dict | None = None,
    ) -> pd.DataFrame:

        default_parameters = {
            "R_ext": 0.005,
            "R_int": 0.01,
            "R_concrete": 0.10,
            "R_ins": 0.32,
            "C_concrete": 2.95e6,
            "C_ins": 3.64e4,
            "alpha": 0.2,
            "S_wall": 7,
            "epsilon": 0.4,
            "fview": 0.5,
            "has_LW_radiation": True
        }
        parameters = {**default_parameters, **property_dict}

        R_ext       = parameters["R_ext"]
        R_int       = parameters["R_int"]
        R_concrete  = parameters["R_concrete"]
        R_ins       = parameters["R_ins"]
        C_concrete  = parameters["C_concrete"]
        C_ins       = parameters["C_ins"]
        alpha       = parameters["alpha"]
        S_wall      = parameters["S_wall"]
        epsilon     = parameters["epsilon"]
        fview       = parameters["fview"]
        has_LW_radiation = parameters["has_LW_radiation"]

        sigma = 5.67e-8  # W/m^2/K^4

        df = simulation_options["dataframe"]
        time  = df["time_sec"].values
        T_ext = df["T_ext"].values
        T_int = df["T_int"].values
        Q_rad = df["Pyr"].values

        startTime = simulation_options.get("startTime", time[0])
        stopTime  = simulation_options.get("stopTime",  time[-1])

        mask  = (time >= startTime) & (time <= stopTime)
        time  = time[mask]
        T_ext = T_ext[mask]
        T_int = T_int[mask]
        Q_rad = Q_rad[mask]

        # init
        T_se        = np.zeros(len(time))
        T_concrete  = np.zeros(len(time))
        T_ins       = np.zeros(len(time))
        T_interface = np.zeros(len(time))
        T_si        = np.zeros(len(time))
        T_sky       = np.zeros(len(time))

        T_se[0]        = T_ext[0]
        T_concrete[0]  = 299
        T_ins[0]       = T_int[0]
        T_interface[0] = (T_ins[0] + T_concrete[0]) / 2
        T_si[0]        = T_int[0]
        T_sky[0]       = T_int[0]

        for t in range(1, len(time)):
            dt = time[t] - time[t - 1]


            ## boolean to turn off long wave radiation exchange with environnement and sky
            T_sky[t] = 0.0552 * (T_ext[t] ** 1.5) if has_LW_radiation else 0.0

            Q_rad_sky = (
                epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_sky[t] ** 4) * S_wall
                if has_LW_radiation else 0.0
            )
            Q_rad_amb = (
                epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_ext[t - 1] ** 4) * S_wall
                if has_LW_radiation else 0.0
            )
            Q_rad_dir = Q_rad[t - 1] * alpha * S_wall

            T_se[t] = (
                T_ext[t - 1] / R_ext
                + T_ins[t - 1] / (R_ins / 2)
                + Q_rad_dir - Q_rad_sky - Q_rad_amb
            ) / (1 / R_ext + 1 / (R_ins / 2))

            T_interface[t] = (
                T_ins[t - 1] / (R_ins / 2) + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / (R_concrete / 2) + 1 / (R_ins / 2))

            T_si[t] = (
                T_int[t - 1] / R_int + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / R_int + 1 / (R_concrete / 2))

            T_ins[t] = T_ins[t - 1] + dt / C_ins * (
                (T_se[t] - T_ins[t - 1]) / (R_ins / 2)
                + (T_interface[t] - T_ins[t - 1]) / (R_ins / 2)
            )

            T_concrete[t] = T_concrete[t - 1] + dt / C_concrete * (
                (T_interface[t] - T_concrete[t - 1]) / (R_concrete / 2)
                + (T_si[t] - T_concrete[t - 1]) / (R_concrete / 2)
            )

        # output
        df_out = pd.DataFrame(
            {
                "T_concrete":  T_concrete,
                "T_interface": T_interface,
                "T_ins":       T_ins,
            },
            index=df.index[mask],
        )
        self.simulation_options = simulation_options
        return df_out

In [ ]:
from corrai.base.parameter import Parameter

mixed_params = [
    Parameter(name="alpha", values=(0.2, 0.4, 0.5), ptype="Choice", model_property="alpha"),
    Parameter(name="epsilon", interval=(0, 1), ptype="Real", model_property="epsilon"),
    Parameter(name="has_LW_radiation", ptype="Binary", model_property="has_LW_radiation"),
]

In [ ]:
from corrai.optimize import Problem, ObjectiveFunction
from corrai.base.metrics import cv_rmse

obj_func = ObjectiveFunction(
    model=OpaqueWallBool(),
    simulation_options=sim_opt,
    parameters=mixed_params,
    indicators_config={
        "T_ins": (cv_rmse, feat_train["T_ins"]),
    },
)

problem = Problem(
    parameters=mixed_params,
    evaluators=[obj_func],
    objective_ids=["T_ins"],
)

In [ ]:
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import RankAndCrowdingSurvival
from pymoo.core.mixed import MixedVariableGA
from pymoo.optimize import minimize

algorithm = MixedVariableGA(pop_size=50, survival=RankAndCrowdingSurvival())
termination = get_termination("n_gen", 5)

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

In [ ]:
parameter_value_pairs = [(p, res.X[p.name]) for p in mixed_params]

model.simulate_parameter(
    parameter_value_pairs=parameter_value_pairs,
    simulation_options=sim_opt,
)